# LinearModel
* TODO document

In [1]:
import SparseArrays: sparse
import NBInclude:@nbinclude
@nbinclude("../Alpha.ipynb")
@nbinclude("EnsembleInputs.ipynb");

## Training

In [2]:
function save_linear_model(alphas, implicit, outdir)
    set_logging_outdir(outdir)
    _, β = regress(alphas, implicit)
    @info "alphas: $alphas"
    @info "coefficients: $β"
    write_params(Dict("β" => β, "alphas" => alphas), outdir)

    splits = []
    @showprogress for split in all_raw_splits
        push!(splits, read_alpha(alphas, split, implicit))
    end
    splits = reduce(cat, splits)
    preds = sparse(splits.user, splits.item, splits.rating)
    write_alpha(preds, [], implicit, outdir; log_test_split = true)
end;

In [3]:
save_linear_model(explicit_raw_alphas, false, "LinearExplicit");

Progress: 100%|███████████████████████████| Time: 0:00:00 (79.46 ns/it)
[ Info: 20220626 11:47:50 alphas: ["ExplicitUserItemBiases", "ExplicitItemCF"]
[ Info: 20220626 11:47:50 coefficients: Float32[0.995312, 0.97463065]
Progress: 100%|█████████████████████████████████████████| Time: 0:00:03
Progress: 100%|███████████████████████████| Time: 0:00:02 ( 0.81 μs/it)
[ Info: 20220626 11:48:14 validation loss: 1.4436618962728862, β: [1.000080799590266]
Progress: 100%|███████████████████████████| Time: 0:00:21 ( 0.75 μs/it)
Progress: 100%|███████████████████████████| Time: 0:00:00 (10.31 ns/it)
[ Info: 20220626 11:48:37 training loss: 1.3773478787995956, β: [1.000080799590266]
Progress: 100%|███████████████████████████| Time: 0:00:02 ( 0.82 μs/it)
Progress: 100%|███████████████████████████| Time: 0:00:00 (31.09 ns/it)
[ Info: 20220626 11:48:41 test loss: 1.4533899806871615, β: [1.000080799590266]
Progress: 100%|███████████████████████████| Time: 0:00:20 ( 0.75 μs/it)
Progress: 100%|██████████

In [4]:
save_linear_model(implicit_raw_alphas, true, "LinearImplicit");

Progress: 100%|███████████████████████████| Time: 0:00:00 (25.39 ns/it)
[ Info: 20220626 11:49:51 alphas: ["UniformBaseline", "PopularityBaseline"]
[ Info: 20220626 11:49:51 coefficients: Float32[1.0087093f-6, 0.9999981, 1.0087035f-6]
Progress: 100%|█████████████████████████████████████████| Time: 0:00:06
Progress: 100%|███████████████████████████| Time: 0:00:03 ( 0.78 μs/it)                                               
[ Info: 20220626 11:50:42 validation loss: 7.37620788263903, β: Float32[1.4704842f-6, 0.99999857]
Progress: 100%|███████████████████████████| Time: 0:00:29 ( 0.75 μs/it)
Progress: 100%|███████████████████████████| Time: 0:00:00 ( 8.41 ns/it)
[ Info: 20220626 11:51:18 training loss: 7.339861048493503, β: Float32[1.4704842f-6, 0.99999857]
Progress: 100%|███████████████████████████| Time: 0:00:03 ( 0.74 μs/it)
Progress: 100%|███████████████████████████| Time: 0:00:00 (23.35 ns/it)
[ Info: 20220626 11:51:22 test loss: 7.371901606347447, β: Float32[1.4704842f-6, 0.99999857